![First Power Bi](.\images\dbatools.png )
# Setting up the containers for the rest of the dbatools notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)


## Create the folders and the credential

Before you run this code, you will have to enter the directory that this notebook is saved in.   

This is a restriction of the Jupyter Notebook in Azure Data Studio as the PowerShell session does not know where it is loc, but I am keeping it for consistency.ted.  

The code below will create a directory called dbachecks-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the $FolderPath but you will have to do this in every notebook.

In [9]:
# Alter the Notebook Directory below before running this code
$notebookDirectory ='C:\Users\mrrob\OneDrive\Documents\GitHub\JupyterNotebooks\notebooks\NotDotNet\dbatools'
$FolderPath = $Env:USERPROFILE + '\Documents\dbatools'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = (Get-Content $notebookDirectory\dockercompose.yml) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content $FolderPath\docker-compose.yml



Creating Directory C:\Users\mrrob\Documents\dbatools
Creating Directory C:\Users\mrrob\Documents\dbatools\SQL1
Creating File C:\Users\mrrob\Documents\dbatools\SQL1\dummyfile.txt
Creating Directory C:\Users\mrrob\Documents\dbatools\SQL2
Creating File C:\Users\mrrob\Documents\dbatools\SQL2\dummyfile.txt
Creating a credential file for the containers - Please don't do this in production
Credential file created
Setting the docker-compose files values


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo).ve.

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run 

`docker pull sqldbawithabeard/dbachecksdemo`


In [10]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatools'
Set-Location $FolderPath
docker-compose up -d

Creating network "dbatools_BeardsMagicNetwork" with the default driver
Creating dbatools_SQL2019_1 ... 
Creating dbatools_SQL2019-1_1 ... 
Creating dbatools_SQL2019_1   ... doneCreating dbatools_SQL2019-1_1 ... done

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way

Otherwise continue to the next notebook

If there are warnings above - Check the containers are running - Look at the status column

In [11]:
docker ps -a

CONTAINER ID        IMAGE                            COMMAND                  CREATED             STATUS              PORTS                                             NAMES
edc37689b392        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   9 seconds ago       Up 8 seconds        0.0.0.0:5023->5023/tcp, 0.0.0.0:15593->1433/tcp   dbatools_SQL2019-1_1
50dd0f84a02b        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   9 seconds ago       Up 8 seconds        0.0.0.0:5022->5022/tcp, 0.0.0.0:15592->1433/tcp   dbatools_SQL2019_1


In [6]:
docker logs dbatools_SQL2019_1

SQL Server 2019 will run as non-root by default.
This container is running as user mssql.
Your master database file is owned by mssql.
To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.
2020-03-26 13:22:25.87 Server      Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 
	Sep 24 2019 13:48:23 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>
2020-03-26 13:22:25.87 Server      UTC adjustment: 0:00
2020-03-26 13:22:25.87 Server      (c) Microsoft Corporation.
2020-03-26 13:22:25.88 Server      All rights reserved.
2020-03-26 13:22:25.88 Server      Server process ID is 32.
2020-03-26 13:22:25.88 Server      Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'.
2020-03-26 13:22:25.88 Server      Registry startup parameters: 
	 -d /var/opt/mssql/data/master.mdf
	 -l /var/opt/mssql/data/mastlog.ldf
	 -e /var/opt/mssql/log/errorlog
2020-03-26 13:22:25.89 Server      SQL Server detected 1 sockets with 2 

In [8]:
docker logs dbatools_SQL2019-1_1

SQL Server 2019 will run as non-root by default.
This container is running as user mssql.
Your master database file is owned by mssql.
To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.
2020-03-26 13:22:25.91 Server      Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 
	Sep 24 2019 13:48:23 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>
2020-03-26 13:22:25.91 Server      UTC adjustment: 0:00
2020-03-26 13:22:25.91 Server      (c) Microsoft Corporation.
2020-03-26 13:22:25.91 Server      All rights reserved.
2020-03-26 13:22:25.91 Server      Server process ID is 32.
2020-03-26 13:22:25.91 Server      Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'.
2020-03-26 13:22:25.91 Server      Registry startup parameters: 
	 -d /var/opt/mssql/data/master.mdf
	 -l /var/opt/mssql/data/mastlog.ldf
	 -e /var/opt/mssql/log/errorlog
2020-03-26 13:22:25.92 Server      SQL Server detected 1 sockets with 2 